In [21]:
#importing required libraries
import yfinance as yf

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import plotly.express as px

from sklearn.linear_model import LinearRegression

#getting historic stock data from yfinance
stocks_list = ['MSFT', 'AAPL','TSLA','^GSPC']

data = yf.download(stocks_list, period='5y',interval='1mo')['Adj Close']
print(data)

data.columns =['MSFT', 'AAPL','TSLA','^GSPC']

[*********************100%%**********************]  4 of 4 completed

Ticker            AAPL        MSFT        TSLA        ^GSPC
Date                                                       
2019-03-01   45.716167  112.473793   18.657333  2834.399902
2019-04-01   48.296192  124.547073   15.912667  2945.830078
2019-05-01   42.134922  117.947754   12.344000  2752.060059
2019-06-01   47.817776  128.224182   14.897333  2941.760010
2019-07-01   51.470787  130.435287   16.107332  2980.379883
2019-08-01   50.431908  131.957214   15.040667  2926.459961
2019-09-01   54.317093  133.520264   16.058001  2976.739990
2019-10-01   60.329151  137.688263   20.994667  3037.560059
2019-11-01   64.813339  145.380829   21.996000  3140.979980
2019-12-01   71.429657  151.965683   27.888666  3230.780029
2020-01-01   75.287560  164.040085   43.371334  3225.520020
2020-02-01   66.494156  156.118958   44.532665  2954.219971
2020-03-01   62.002235  152.390457   34.933334  2584.590088
2020-04-01   71.635757  173.165253   52.125332  2912.429932
2020-05-01   77.521698  177.069000   55.

In [22]:
#Normalizing Stock Prices

def normalize_prices(df):

    df_ = df.copy()

    for stock in df_.columns:

        df_[stock] = df_[stock]/df_[stock][0]

    return df_

norm_df = normalize_prices(data)

# Plotting the normalized stock prices

fig = px.line(title = "Normalized stock prices")

for stock in norm_df.columns:

    fig.add_scatter(x = norm_df.index.values, y = norm_df[stock], name = stock)

fig.show()

/tmp/ipykernel_86655/949410128.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [23]:
# Calculating Daily % change in stock prices

daily_returns = norm_df.pct_change()

daily_returns.iloc[0,:] = 0

# Boxplot of daily returns (in %)

daily_returns.boxplot(figsize=(6, 5), grid=False)

plt.title("Daily returns of the stocks")

Text(0.5, 1.0, 'Daily returns of the stocks')

In [ ]:
# Initializing empty dictionaries to save results

beta,alpha = dict(), dict()

# Make a subplot

fig, axes = plt.subplots(1,3, dpi=150, figsize=(15,8))

axes = axes.flatten()

# Loop on every daily stock return

for idx, stock in enumerate(daily_returns.columns.values[:-1]):

    # scatter plot between stocks and the NSE

    daily_returns.plot(kind = "scatter", x = "^GSPC", y = stock, ax=axes[idx])

    # Fit a line (regression using polyfit of degree 1)

    b_, a_ = np.polyfit(daily_returns["^GSPC"] ,daily_returns[stock], 1)
    print(b_, a_)

    regression_line = b_ * daily_returns["^GSPC"] + a_

    axes[idx].plot(daily_returns["^GSPC"], regression_line, "-", color = "r")

    # save the regression coeeficient for the current stock

    beta[stock] = b_

    alpha[stock] = a_

plt.suptitle("Beta estimation: regression between ^GSPC and individual stock daily performance", size=20)

plt.show()

In [18]:
keys = list(beta.keys()) # list of stock names

beta_3 = dict()

for k in keys:
    beta_3[k] = [daily_returns[[k,'^GSPC']].cov()/daily_returns['^GSPC'].var()][0].iloc[0,1]
    print(f"el beta de {k}: {beta[k]}")

el beta de MSFT: 1.3042928582581823
el beta de AAPL: 0.892620582607367
el beta de TSLA: 2.427022352589082


In [12]:
# Initialize the expected return dictionary

ER = dict()

rf = 0.0733

trading_days = 250

# Estimate the expected return of the market using the daily returns

rm = daily_returns["^GSPC"].mean() * trading_days

for k in keys:

    # Calculate return for every security using CAPM

    ER[k] = rf + beta[k] * (rm-rf)

for k in keys:

    print("Expected return based on CAPM model for {} is {}%".format(k, round(ER[k]*100, 2)))

# Calculating historic returns

for k in keys:

    print('Return based on historkeysical data for {} is {}%'.format(k, round(daily_returns[k].mean() * 100 * trading_days, 2)))

Expected return based on CAPM model for MSFT is 354.99%
Expected return based on CAPM model for TSLA is 245.26%
Expected return based on CAPM model for AAPL is 654.26%
Return based on historkeysical data for MSFT is 684.35%
Return based on historkeysical data for TSLA is 598.23%
Return based on historkeysical data for AAPL is 1526.55%
